In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family = font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

# 동명칭 통일

## (1) 기준 : 유동인구

In [4]:
pop1=pd.read_csv("../../Data/회귀분석/2018~2022유동인구.csv")

In [5]:
pop2=pd.read_csv("../../Data/회귀분석/2018~2022직장인구.csv")

In [6]:
pop3=pd.read_csv("../../Data/회귀분석/2018~2022거주인구.csv")

In [7]:
flow=pd.merge(pop1, pop2, on=["구","동","연도"], how="outer")

In [8]:
flow=pd.merge(flow, pop3, on=["구","동","연도"], how="outer")

In [9]:
del flow["Unnamed: 0_x"]
del flow["Unnamed: 0_y"]
del flow["Unnamed: 0"]

In [10]:
flow22=flow[flow["연도"]==2022]
flow22.head()

,구,동,평균유동인구,연도,평균직장인구,평균거주인구
1696,종로구,청운효자동,13676.833333,2022,44.0,51.0
1697,종로구,사직동,21816.666667,2022,410.0,82.0
1698,종로구,삼청동,4547.833333,2022,42.0,19.0
1699,종로구,부암동,4436.166667,2022,19.0,45.0
1700,종로구,평창동,1126.500000,2022,28.0,22.0
...,...,...,...,...,...,...
2115,강동구,성내2동,150514.500000,2022,37.0,389.0
2116,강동구,성내3동,127091.166667,2022,72.0,365.0
2117,강동구,길동,100750.833333,2022,60.0,293.0
2118,강동구,둔촌1동,9407.833333,2022,17.0,0.0


In [12]:
flow_dong=set(flow22["동"])
dong_std=list(flow_dong)

In [15]:
gu=list(set(flow22["구"].unique()))
print(len(gu))

25


## (1) 자동차등록대수

In [46]:
car= pd.read_csv("../../Data/회귀분석/동별차량등록대수.csv")
car.columns=["연도","구별","동별","차량등록대수"]

In [49]:
car=car[car["동별"]!="기타"]

In [52]:
car.loc[car["동별"]=="명륜3가동","동별"]="혜화동"
car.loc[car["동별"]=="공릉1.3동","동별"]="공릉1동"
car.loc[car["동별"].str.contains("도화"),"동별"]="도화동"
car.loc[car["동별"]=="독산1동분소","동별"]="독산1동"
car.loc[car["동별"]=="길1동","동별"]="길동"
car.loc[car["동별"]=="하일동","동별"]="강일동"
car.loc[car["동별"].str.contains("상일"),"동별"]="상일동"
car.loc[car["동별"]=="남대문로5가동","동별"]="회현동"
car.loc[car["동별"]=="신당1동","동별"]="신당동"
car.loc[car["동별"]=="신당2동","동별"]="신당동"
car.loc[car["동별"]=="신당3동","동별"]="신당동"
car.loc[car["동별"]=="신당4동","동별"]="신당동"
car.loc[car["동별"]=="신당6동","동별"]="신당동"
car.loc[car["동별"]=="을지로3.4.5가동","동별"]="을지로동"
car.loc[car["동별"]=="영등포1동","동별"]="영등포동"
car.loc[car["동별"]=="태평로1가동","동별"]="명동"

In [ ]:
print(car.loc[car["동별"]=="노고산동","동별"])
print(car.loc[car["동별"]=="도곡동","동별"])
print(car.loc[car["동별"]=="논현동","동별"])
print(car.loc[car["동별"]=="월계4동","동별"])
print(car.loc[car["동별"]=="평택시"])
print(car.loc[car["동별"]=="개봉본동","동별"])
print(car.loc[car["동별"]=="구로본동","동별"])
print(car.loc[car["동별"]=="항동","동별"])

In [56]:
car.drop([218,687,1158,1633,2107,
          1193,1668,2142,
          6,475,944,1418,1892,
          163,632,1103,1578,2052,
          2266,
          122,591,1061,1536,2010,
          130,599,1069,1544,2018,
          1075, 1550, 2024], inplace=True)

In [57]:
car22=car[car["연도"]==2022]

In [58]:
for i in gu : 
    std_c=flow22.loc[flow22["구"]== i,["동"]].sort_values(by="동")
    con_c=car22.loc[car22["구별"]== i,["동별"]].sort_values(by="동별")
    df_c=pd.merge(std_c,con_c,left_on="동", right_on="동별", how="outer")
    df_c["동"]=df_c["동"].astype(str)
    df_c["동별"]=df_c["동별"].astype(str)
    df_c["판단"]=""
    for j in df_c.index :
        if df_c["동"][j].strip() != df_c["동별"][j].strip() :
            df_c["판단"][j]="수정"
        else : 
            df_c["판단"][j]="pass"
    if df_c["판단"].str.contains("수정").any() :
        print(i, df_c[df_c["판단"]=="수정"])
        print("---------------------------------------------------")
    else :
        print(i)
        print("---------------------------------------------------")

마포구
---------------------------------------------------
서초구
---------------------------------------------------
송파구
---------------------------------------------------
중랑구
---------------------------------------------------
강남구
---------------------------------------------------
성북구
---------------------------------------------------
관악구
---------------------------------------------------
동작구
---------------------------------------------------
노원구
---------------------------------------------------
양천구
---------------------------------------------------
용산구
---------------------------------------------------
광진구
---------------------------------------------------
서대문구
---------------------------------------------------
도봉구
---------------------------------------------------
중구
---------------------------------------------------
성동구
---------------------------------------------------
동대문구
---------------------------------------------------
은평구
-------------------------------------------

In [59]:
car=car.groupby(["구별","동별","연도"])[["차량등록대수"]].sum()

In [60]:
car.to_csv("../../Data/회귀분석/동별차량(수정).csv",encoding="utf-8")

## (2) 면적

In [295]:
area=pd.read_csv("../../Data/회귀분석/동별면적.csv")

In [296]:
area.loc[area["동별"].str.contains("상일"),"동별"]="상일동"

In [292]:
# print(area.loc[area["동별"]=="일원2동","동별"])
print(area.loc[area["동별"]=="개포3동","동별"])
print(area.loc[area["동별"]=="항동","동별"])
area.drop([2058,
          1123,1548,1974], inplace=True)

2058    개포3동
Name: 동별, dtype: object
1123    항동
1548    항동
1974    항동
Name: 동별, dtype: object


In [300]:
area22=area[area["연도"]==2022]

In [301]:
for i in gu : 
    std_p=flow22.loc[flow22["구"]== i,["동"]].sort_values(by="동")
    con_p=area22.loc[area22["구별"]== i,["동별"]].sort_values(by="동별")
    df_p=pd.merge(std_p,con_p,left_on="동", right_on="동별", how="outer")
    df_p["동"]=df_p["동"].astype(str)
    df_p["동별"]=df_p["동별"].astype(str)
    df_p["판단"]=""
    for j in df_p.index :
        if df_p["동"][j].strip() != df_p["동별"][j].strip() :
            df_p["판단"][j]="수정"
        else : 
            df_p["판단"][j]="pass"
    if df_p["판단"].str.contains("수정").any() :
        print(i, df_p[df_p["판단"]=="수정"])
        print("---------------------------------------------------")
    else :
        print(i)
        print("---------------------------------------------------")

강
---------------------------------------------------
동
---------------------------------------------------
구
---------------------------------------------------


In [89]:
area=area.groupby(["구별","동별","연도"])[["면적"]].sum()

In [90]:
area.to_csv("../../Data/회귀분석/동별면적(수정).csv", encoding="utf-8")

## (4) 인구밀도

In [91]:
p_den=pd.read_csv("../../Data/회귀분석/인구밀도_20230424093342.csv", usecols=[0,2,3,6])
p_den.columns=["연도","구별","동별","인구밀도"]

In [92]:
p_den=p_den[(p_den["구별"]!="소계")& (p_den["동별"]!="소계")]

In [98]:
p_den.loc[p_den["동별"].str.contains("상일"),"동별"]="상일동"

In [102]:
print(p_den.loc[p_den["동별"]=="개포3동","동별"])
print(p_den.loc[p_den["동별"]=="항동","동별"])
p_den.drop([2186,
          1193, 1644, 2096], inplace=True)

2186    개포3동
Name: 동별, dtype: object
1193    항동
1644    항동
2096    항동
Name: 동별, dtype: object


In [103]:
p_den22=p_den[p_den["연도"]==2022]
p_den22

,연도,구별,동별,인구밀도
1805,2022,종로구,사직동,7606
1806,2022,종로구,삼청동,1773
1807,2022,종로구,부암동,4201
1808,2022,종로구,평창동,2013
1809,2022,종로구,무악동,22367
...,...,...,...,...
2250,2022,강동구,천호2동,21545
2251,2022,강동구,길동,20910
2252,2022,강동구,강일동,14627
2253,2022,강동구,상일동,14784


In [104]:
for i in gu : 
    std_d=flow22.loc[flow22["구"]== i,["동"]].sort_values(by="동")
    con_d=p_den22.loc[p_den22["구별"]== i,["동별"]].sort_values(by="동별")
    df_d=pd.merge(std_d,con_d,left_on="동", right_on="동별", how="outer")
    df_d["동"]=df_d["동"].astype(str)
    df_d["동별"]=df_d["동별"].astype(str)
    df_d["판단"]=""
    for j in df_d.index :
        if df_d["동"][j].strip() != df_d["동별"][j].strip() :
            df_d["판단"][j]="수정"
        else : 
            df_d["판단"][j]="pass"
    if df_d["판단"].str.contains("수정").any() :
        print(i, df_d[df_d["판단"]=="수정"])
        print("---------------------------------------------------")
    else :
        print(i)
        print("---------------------------------------------------")

마포구
---------------------------------------------------
서초구
---------------------------------------------------
송파구
---------------------------------------------------
중랑구
---------------------------------------------------
강남구        동   동별  판단
19  일원2동  nan  수정
---------------------------------------------------
성북구
---------------------------------------------------
관악구
---------------------------------------------------
동작구
---------------------------------------------------
노원구
---------------------------------------------------
양천구
---------------------------------------------------
용산구
---------------------------------------------------
광진구
---------------------------------------------------
서대문구
---------------------------------------------------
도봉구
---------------------------------------------------
중구
---------------------------------------------------
성동구
---------------------------------------------------
동대문구
---------------------------------------------------
은평구
-------

In [105]:
p_den.to_csv("../../Data/회귀분석/인구밀도.csv", encoding="utf-8")

## (5) 교통량 

In [257]:
traffic=pd.read_csv("../../Data/회귀분석/교통량_18_22.csv",
            header=None)

In [258]:
traffic.columns=["구별", "동별","연도","평균교통량"]

In [259]:
# t22=traffic[traffic["연도"]==2022]

In [285]:
df_m=traffic.groupby(["연도","구별"])[["평균교통량"]].mean()
df_m.reset_index(inplace=True)

In [287]:
df_m[(df_m["구별"]=="관악구") & (df_m["연도"]==2022)]

,연도,구별,평균교통량
99,2022,관악구,1044.853917


In [261]:
df_t=df_m.groupby("연도")[["평균교통량"]].mean()
df_t.reset_index(inplace=True)

In [282]:
df_std=flow[["구","동","연도"]]

,구,동,연도
318,관악구,보라매동,2018
319,관악구,청림동,2018
320,관악구,성현동,2018
321,관악구,행운동,2018
322,관악구,낙성대동,2018
...,...,...,...
2030,관악구,조원동,2022
2031,관악구,대학동,2022
2032,관악구,삼성동,2022
2033,관악구,미성동,2022


In [268]:
m=pd.merge(df_std, traffic,left_on=["동","연도"], right_on=["동별","연도"], how="outer")
# m
m=pd.merge(m, df_m,left_on=["구","연도"], right_on=["구별","연도"], how="outer")
m

,구,동,연도,구별_x,동별,평균교통량_x,구별_y,평균교통량_y
0,종로구,청운효자동,2018,종로구,청운효자동,641.593260,종로구,985.40366
1,종로구,사직동,2018,종로구,사직동,990.459829,종로구,985.40366
2,종로구,삼청동,2018,NaN,NaN,NaN,종로구,985.40366
3,종로구,부암동,2018,NaN,NaN,NaN,종로구,985.40366
4,종로구,평창동,2018,종로구,평창동,932.114484,종로구,985.40366
...,...,...,...,...,...,...,...,...
2140,NaN,NaN,2022,강남구,개포3동,2728.373885,NaN,NaN
2141,NaN,NaN,2022,강동구,상일2동,2015.964228,NaN,NaN
2142,NaN,NaN,2022,광진구,자양동,2441.924718,NaN,NaN
2143,NaN,NaN,2022,구로구,궁동,851.737562,NaN,NaN


In [269]:
m.loc[m["동"].isna(),"동별"].unique()

array(['개포3동', '상일2동', '자양동', '궁동', '서초동'], dtype=object)

In [270]:
m=m.loc[~ m["동별"].isin(['개포3동', '상일2동', '자양동', '궁동', '서초동'])]

### 결측치

- 구별 평균교통량 대체
- 강북구의 경우 지점이 없어 구별 평균교통량의 평균으로 대체

In [271]:
m[m["평균교통량_x"].isna()]

,구,동,연도,구별_x,동별,평균교통량_x,구별_y,평균교통량_y
2,종로구,삼청동,2018,NaN,NaN,NaN,종로구,985.403660
3,종로구,부암동,2018,NaN,NaN,NaN,종로구,985.403660
5,종로구,무악동,2018,NaN,NaN,NaN,종로구,985.403660
7,종로구,가회동,2018,NaN,NaN,NaN,종로구,985.403660
9,종로구,종로5.6가동,2018,NaN,NaN,NaN,종로구,985.403660
...,...,...,...,...,...,...,...,...
2115,강동구,성내2동,2022,NaN,NaN,NaN,강동구,2124.166204
2116,강동구,성내3동,2022,NaN,NaN,NaN,강동구,2124.166204
2117,강동구,길동,2022,NaN,NaN,NaN,강동구,2124.166204
2118,강동구,둔촌1동,2022,NaN,NaN,NaN,강동구,2124.166204


In [275]:
for i in m.index :
    if np.isnan(m.loc[i, "평균교통량_x"]) :
        year= m.loc[i, "연도"]
        gu = m.loc[i, "구"]
        dong = m.loc[i, "동"]
        m.loc[i, "구별_x"]=gu
        m.loc[i, "동별"]=dong       
        if gu != "강북구" :
            m.loc[i, "평균교통량_x"] = m.loc[i, "평균교통량_y"]
        else :
            m.loc[i, "평균교통량_x"]=df_t.loc[df_t["연도"]==year,"평균교통량"].values
    else :  pass

In [274]:
df_t.loc[df_t["연도"]==2022,]

,연도,평균교통량
4,2022,1569.546583


In [276]:
m.loc[m["구"]=="강북구","평균교통량_x"]

130     1712.090976
131     1712.090976
132     1712.090976
133     1712.090976
134     1712.090976
           ...     
1834    1569.546583
1835    1569.546583
1836    1569.546583
1837    1569.546583
1838    1569.546583
Name: 평균교통량_x, Length: 65, dtype: float64

In [280]:
m.loc[m["평균교통량_x"].isna()]

,구,동,연도,구별_x,동별,평균교통량_x,구별_y,평균교통량_y


In [277]:
tm=m[["구별_x","동별","연도","평균교통량_x"]]

In [278]:
tm.rename(columns={"구별_x":"구별","평균교통량_x":"평균교통량"}, inplace=True)

C:\Users\acorn\AppData\Local\Temp\ipykernel_20456\3221923978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tm.rename(columns={"구별_x":"구별","평균교통량_x":"평균교통량"}, inplace=True)


In [279]:
tm.to_csv("../../Data/회귀분석/교통량(수정).csv", encoding="utf-8")

## (4) 승객수

In [303]:
passenger=pd.read_csv("../../../GwangProject/Data/회귀분석/동별대중교통이용승객수.csv", 
                encoding= "utf-8")

In [307]:
del passenger["Unnamed: 0"]

In [308]:
passenger.columns=["구별","동별","승객수"]

In [315]:
gu=list(set(flow22["구"].unique()))
print(len(gu))

for i in gu : 
    std_ps=flow22.loc[flow22["구"]== i,["동"]].sort_values(by="동")
    con_ps=passenger.loc[passenger["구별"]== i,["동별"]].sort_values(by="동별")
    df_ps=pd.merge(std_ps,con_ps,left_on="동", right_on="동별", how="outer")
    df_ps["동"]=df_ps["동"].astype(str)
    df_ps["동별"]=df_ps["동별"].astype(str)
    df_ps["판단"]=""
    for j in df_ps.index :
        if df_ps["동"][j].strip() != df_ps["동별"][j].strip() :
            df_ps["판단"][j]="수정"
        else : 
            df_ps["판단"][j]="pass"
    if df_ps["판단"].str.contains("수정").any() :
        print(i, df_ps[df_ps["판단"]=="수정"])
        print("---------------------------------------------------")
    else :
        print(i)
        print("---------------------------------------------------")

25
마포구
---------------------------------------------------
서초구
---------------------------------------------------
송파구
---------------------------------------------------
중랑구
---------------------------------------------------
강남구
---------------------------------------------------
성북구
---------------------------------------------------
관악구
---------------------------------------------------
동작구
---------------------------------------------------
노원구
---------------------------------------------------
양천구
---------------------------------------------------
용산구
---------------------------------------------------
광진구
---------------------------------------------------
서대문구
---------------------------------------------------
도봉구
---------------------------------------------------
중구
---------------------------------------------------
성동구
---------------------------------------------------
동대문구
---------------------------------------------------
은평구
----------------------------------------

In [314]:
passenger["동별"]=passenger["동별"].str.replace(",",".")

In [318]:
passenger.to_csv("../../Data/회귀분석/동별대중교통이용승객수(수정).csv", 
                encoding= "utf-8")